# 📘 P1.2.3.5 – Python File Operations
## Topic: File Error Handling

## 🎯 Learning Objectives
By the end of this notebook, you will:
- Handle common file-related exceptions
- Use try-except-finally with file operations
- Implement graceful error recovery
- Validate file operations before executing
- Log file errors for debugging
- Build robust file processing pipelines

## ⚠️ Why File Error Handling Matters
File operations fail for many reasons:
- File doesn't exist
- No permission to read/write
- Disk is full
- File is locked by another program
- Invalid file format

**Without proper error handling, your program crashes!**

## 🔴 Common File Exceptions

| Exception | Cause |
|---|---|
| `FileNotFoundError` | File doesn't exist |
| `PermissionError` | No permission to access file |
| `IsADirectoryError` | Tried to open directory as file |
| `IOError` | General input/output error |
| `OSError` | Operating system error |

## 📂 Setup: Create Data Directory

In [ ]:
from pathlib import Path

# Create data directory
Path("data").mkdir(exist_ok=True)

# Create a sample file
Path("data/example.txt").write_text("Hello, World!")

print("✅ Setup complete")

## 1️⃣ Handling FileNotFoundError

In [ ]:
# ❌ BAD: No error handling - Program crashes
try:
    with open("data/missing.txt", "r") as f:
        content = f.read()
except FileNotFoundError:
    print("This would crash without try-except!")

print("\n" + "="*50)

# ✅ GOOD: Handle the error gracefully
try:
    with open("data/missing.txt", "r") as f:
        content = f.read()
        print(content)
except FileNotFoundError:
    print("⚠️ Error: File 'data/missing.txt' not found!")
    print("Creating a default file...")
    with open("data/missing.txt", "w") as f:
        f.write("Default content")
    print("✅ Default file created")

## 2️⃣ Checking File Existence Before Opening

In [ ]:
from pathlib import Path

def safe_read_file(filepath):
    """Read file only if it exists"""
    path = Path(filepath)
    
    if not path.exists():
        print(f"❌ Error: {filepath} does not exist")
        return None
    
    if not path.is_file():
        print(f"❌ Error: {filepath} is not a file")
        return None
    
    try:
        with open(filepath, "r") as f:
            return f.read()
    except Exception as e:
        print(f"❌ Error reading file: {e}")
        return None

# Test the function
content = safe_read_file("data/example.txt")
if content:
    print(f"✅ File content: {content}")

# Try with non-existent file
safe_read_file("data/nonexistent.txt")

## 3️⃣ Handling Multiple Exceptions

In [ ]:
def robust_file_reader(filepath):
    """Handle multiple possible errors"""
    try:
        with open(filepath, "r") as f:
            return f.read()
    except FileNotFoundError:
        print(f"❌ File not found: {filepath}")
        return None
    except PermissionError:
        print(f"❌ Permission denied: {filepath}")
        return None
    except IsADirectoryError:
        print(f"❌ Cannot read directory as file: {filepath}")
        return None
    except UnicodeDecodeError:
        print(f"❌ File is not a text file: {filepath}")
        return None
    except Exception as e:
        print(f"❌ Unexpected error: {type(e).__name__} - {e}")
        return None

# Test with various scenarios
print("Test 1: Valid file")
robust_file_reader("data/example.txt")

print("\nTest 2: Directory instead of file")
robust_file_reader("data")

print("\nTest 3: Non-existent file")
robust_file_reader("data/xyz.txt")

## 4️⃣ Using finally for Cleanup

In [ ]:
def process_file_with_cleanup(filepath):
    """Demonstrate try-except-finally"""
    file = None
    try:
        print(f"Opening {filepath}...")
        file = open(filepath, "r")
        content = file.read()
        print(f"✅ Read {len(content)} characters")
        return content
    except FileNotFoundError:
        print(f"❌ File not found: {filepath}")
        return None
    finally:
        # This ALWAYS runs, even if there's an error
        if file and not file.closed:
            file.close()
            print("🔒 File closed in finally block")

# Test with existing file
print("Test 1: Existing file")
process_file_with_cleanup("data/example.txt")

print("\n" + "="*50 + "\n")

# Test with missing file
print("Test 2: Missing file")
process_file_with_cleanup("data/nope.txt")

## 5️⃣ Validating File Operations

In [ ]:
import os
from pathlib import Path

def safe_write_file(filepath, content):
    """Write file with validation"""
    path = Path(filepath)
    
    # Check if parent directory exists
    if not path.parent.exists():
        print(f"⚠️ Creating directory: {path.parent}")
        path.parent.mkdir(parents=True, exist_ok=True)
    
    # Check write permissions (on parent directory)
    if not os.access(path.parent, os.W_OK):
        print(f"❌ No write permission for {path.parent}")
        return False
    
    try:
        with open(filepath, "w") as f:
            f.write(content)
        print(f"✅ Successfully wrote to {filepath}")
        return True
    except Exception as e:
        print(f"❌ Error writing file: {e}")
        return False

# Test the function
safe_write_file("data/output/results.txt", "Test results")
safe_write_file("data/test.txt", "Another test")

## 🔄 Error Recovery Strategies

| Strategy | When to Use | Example |
|---|---|---|
| **Default Value** | Non-critical file | Return empty dict if config missing |
| **Create Missing File** | Expected to exist | Create default config file |
| **Skip and Continue** | Batch processing | Process remaining files |
| **Retry with Backoff** | Temporary issues | Wait and retry locked files |
| **User Prompt** | Interactive apps | Ask user to provide file |
| **Fail Fast** | Critical files | Raise error immediately |

### ✅ Key Takeaways
- File operations are **inherently risky** - always use error handling
- Use **specific exceptions** (FileNotFoundError, PermissionError) instead of generic Exception
- **Check before you act** - validate existence and permissions
- **Log errors** to help with debugging and monitoring
- **Graceful degradation** - provide fallbacks and defaults
- Combine `try-except-finally` for guaranteed cleanup
- **In AI/ML:** Handle missing data files, corrupt model files, insufficient disk space for large datasets, permission issues in shared environments